In [2]:
import sys 
import os
sys.path.append(os.path.abspath("/home/scher/Repos/servier_drug_prop_classification"))
import numpy as np
import pandas as pd

In [51]:
from sklearn.model_selection import train_test_split

In [18]:
# we want to use the smile string directly into the training, it means the preprocessing should not use fingerprint_features to process raw data
# instead, let's do One Hot Encoding on every smile.

data = pd.read_csv('../src/data/dataset_single.csv')


In [22]:
# concatenating all smiles together
all_smiles = ''
for idx, smile in enumerate(data["smiles"]):
    all_smiles = all_smiles + smile

In [27]:
# getting unique chars
strs = list(all_smiles)
concatenated = ''.join(strs)
unique_chars = ''.join(sorted(set(concatenated), key=concatenated.index))
print(unique_chars)

Cc1(N2=O)345nSo/[H]-ls#F\+Br6


In [66]:
# for each unique char, give an int value for encoding
char_to_int = dict((c, i + 1) for i, c in enumerate(unique_chars))
int_to_char = dict((i + 1, c) for i, c in enumerate(unique_chars))
integer_encoded = [char_to_int[char] for char in data["smiles"][0]]
print(integer_encoded)


[1, 2, 3, 2, 2, 2, 2, 4, 5, 6, 1, 1, 5, 4, 1, 4, 7, 8, 9, 1, 10, 11, 1, 1, 12, 1, 1, 4, 1, 1, 4, 1, 12, 9, 1, 10, 9, 1, 11, 9, 1, 1, 6, 9, 2, 3, 1]


In [61]:
data["smile_len"] = data["smiles"].apply(len)
smile_max_length = max(data["smile_len"])


74


In [80]:
# one hot encoding
def one_hot_encoding(smile, char_to_int, unique_chars):
    onehot_encoded = list()
    # encoding the smile using the char_to_int dict
    integer_encoded = [char_to_int[char] for char in smile]
    # Adding padding for every smile length to match
    current_length = len(integer_encoded)
    if current_length < 74:
        zeros_to_add = 74 - current_length
        integer_encoded.extend([0] * zeros_to_add)
    # one hot encoding
    for value in integer_encoded:
        letter = [0 for _ in range(len(unique_chars))]
        if value != 0:
            print(value)
            letter[value - 1] = 1
        onehot_encoded.append(letter)
    return np.array(onehot_encoded)

data["encoded"] = data["smiles"].apply(lambda x : one_hot_encoding(x, char_to_int, unique_chars))


1
2
3
2
2
2
2
4
5
6
1
1
5
4
1
4
7
8
9
1
10
11
1
1
12
1
1
4
1
1
4
1
12
9
1
10
9
1
11
9
1
1
6
9
2
3
1
1
13
3
2
2
13
2
3
14
1
1
4
7
8
9
5
2
3
2
2
2
4
8
2
6
2
2
2
2
2
6
9
2
2
3
1
8
2
3
2
2
6
2
4
2
2
3
5
1
4
7
8
9
1
5
3
1
4
7
8
9
5
1
10
4
1
1
2
11
2
2
2
2
2
11
10
9
1
3
7
8
9
15
2
3
2
2
2
2
2
3
6
8
7
1
3
16
1
4
7
1
16
5
1
6
1
1
14
4
7
8
9
4
7
8
9
1
6
9
2
6
2
2
2
2
2
6
1
4
7
8
9
5
3
2
3
2
2
2
2
2
3
5
1
4
7
8
9
5
1
4
1
2
3
2
2
2
2
2
3
9
1
4
7
8
9
8
1
1
13
3
2
4
1
14
2
6
13
2
2
13
6
1
9
13
2
6
2
2
4
1
4
7
8
9
8
9
2
2
2
6
3
1
1
2
3
2
2
2
2
6
2
3
5
1
4
7
8
9
1
6
3
1
6
1
4
7
8
9
5
4
1
2
10
2
2
2
2
2
10
9
1
4
7
8
9
1
6
1
6
1
1
1
5
6
3
1
8
2
3
2
2
2
4
1
6
1
4
1
4
7
8
9
5
1
2
10
2
2
2
2
2
10
9
7
1
4
1
9
5
7
1
10
5
7
1
5
5
10
6
9
2
2
3
8
1
1
1
1
2
3
2
2
4
7
8
9
13
2
4
14
1
1
4
7
8
9
5
4
1
1
4
1
9
1
9
1
6
1
1
14
4
7
8
9
4
7
8
9
1
6
9
17
13
18
19
3
1
1
13
3
2
13
2
6
2
3
2
4
7
8
9
13
4
1
1
4
7
8
9
5
2
3
2
2
4
1
9
15
13
3
9
2
4
7
8
9
13
6
1
2
3
2
2
2
2
2
3
1
2
3
2
2
2
2
4
5
1
4
7
8
9
5
6
1
1
1
4
2
10
13
2

In [81]:
data["encoded_shape"] = data["encoded"].apply(lambda x : x.shape)

In [82]:
print(data["encoded_shape"])


0       (74, 29)
1       (74, 29)
2       (74, 29)
3       (74, 29)
4       (74, 29)
          ...   
4994    (74, 29)
4995    (74, 29)
4996    (74, 29)
4997    (74, 29)
4998    (74, 29)
Name: encoded_shape, Length: 4999, dtype: object


In [94]:
X = np.stack(data["encoded"], axis=0)
y = np.stack(data["P1"], axis=0)

In [95]:
print(X.shape)

(4999, 74, 29)


In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [97]:
X_train.shape

(3999, 74, 29)

In [98]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(LSTM(units=100, input_shape=(74,29), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size=64)

2023-10-22 23:06:37.804015: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 68654832 exceeds 10% of free system memory.


Epoch 1/3
63/63 [==============================] - 9s 83ms/step - loss: 0.4958 - accuracy: 0.8217
Epoch 2/3
63/63 [==============================] - 5s 80ms/step - loss: 0.4710 - accuracy: 0.8217
Epoch 3/3
63/63 [==============================] - 5s 84ms/step - loss: 0.4695 - accuracy: 0.8217


In [99]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 82.20%


In [9]:
from src.scripts.preprocess import smiles_to_numpy, morganFP_to_numpy, one_hot_encoding
import pickle
import tensorflow as tf

X_test = 'Cn1ccnc1SCC(=O)Nc1ccc(Oc2ccccc2)cc1'
model_folder = "../src/models/model_lively_LSTM"
preprocess_params = pickle.load(open(model_folder + "/preprocess_params.pkl", 'rb'))
model = tf.keras.models.load_model(model_folder + '/LSTM_model.h5')
X_test = one_hot_encoding(X_test, preprocess_params["char_to_int"], preprocess_params["char_to_int"].keys(),  preprocess_params["smile_max_length"])


2023-10-23 12:23:54.400090: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 12:23:54.476968: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 12:23:54.477027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 12:23:54.477101: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 12:23:54.490793: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 12:23:54.492701: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

ValueError: in user code:

    File "/home/scher/miniconda/envs/servier/lib/python3.9/site-packages/keras/src/engine/training.py", line 2416, in predict_function  *
        return step_function(self, iterator)
    File "/home/scher/miniconda/envs/servier/lib/python3.9/site-packages/keras/src/engine/training.py", line 2401, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/scher/miniconda/envs/servier/lib/python3.9/site-packages/keras/src/engine/training.py", line 2389, in run_step  **
        outputs = model.predict_step(data)
    File "/home/scher/miniconda/envs/servier/lib/python3.9/site-packages/keras/src/engine/training.py", line 2357, in predict_step
        return self(x, training=False)
    File "/home/scher/miniconda/envs/servier/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/scher/miniconda/envs/servier/lib

In [16]:
X_test = np.array([X_test])
print(X_test.shape)
res = model.predict(X_test)

(1, 74, 29)
1/1 [==============================] - 1s 597ms/step


In [21]:
print(res[0][0] > 0.5)

True
